In [1]:
import tensorflow as tf
import os

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Convolutional Layer

```python
"""
input: N x h x w x d 
  N: batch size
  h: height
  w: width
  d: depth (channel)

filter: e x e x d x k
  e: filter (or kernel) width / height, 3 x 3 or 5 x 5
  d: depth is the same with input's d
  k: kernel sizes
  
padding:
  SAME: with zero padding
  VALID: without padding
"""
tf.nn.conv2d(input=input, \
             filter=filter, \
             strides=strides, \
             padding=padding, \
             use_cudnn_on_gpu=True, \
             name=None)
```

# Pooling Layer

## Max Pooling

```python
"""
value: N x w x h x d

ksize: [1, k, k, 1]
  k: kernel size

strides: [1, s, s, 1]
  s: stride size
  
padding: SAME or VALID
"""
tf.nn.max_pool(value=input, \
               ksize=[1,k,k,1], \
               strides=[1,s,s,1], \
               padding="SAME", \
               name=None)
```

# Padding

In [2]:
# SAME vs. VALID
input = tf.Variable(tf.random_normal([1,5,5,5]))
filter = tf.Variable(tf.random_normal([3,3,5,7]))
conv = tf.nn.conv2d(input=input, filter=filter, strides=[1,1,1,1], padding="SAME")
print(conv)

Tensor("Conv2D:0", shape=(1, 5, 5, 7), dtype=float32)


# MNIST Example

In [3]:
def conv2d(input, weight_shape, bias_shape):
    in_count = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=(2.0/in_count)**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    conv_out = tf.nn.conv2d(input, W, strides=[1,1,1,1], padding="SAME")
    return tf.nn.relu(tf.nn.bias_add(conv_out, b))

In [4]:
def max_pool(input, k=2):
    return tf.nn.max_pool(value=input, ksize=[1,k,k,1], strides=[1,k,k,1], padding="SAME")

In [5]:
def layer(input, weight_shape, bias_shape):
    in_count = weight_shape[0] * weight_shape[1]
    w_init = tf.random_normal_initializer(stddev=(2.0/in_count)**0.5)
    b_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=w_init)
    b = tf.get_variable("b", bias_shape, initializer=b_init)
    return tf.nn.relu(tf.matmul(input, W) + b)

In [6]:
def loss(output, y):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=output, labels=y)
    return tf.reduce_mean(cross_entropy)

In [7]:
def inference(input, keep_prob=0.5):
    x = tf.reshape(input, [-1, 28, 28, 1])
    
    with tf.variable_scope("conv_1"):
        conv_1 = conv2d(x, [5, 5, 1, 32], [32])                # conv_1: 28 x 28 x 32
        pool_1 = max_pool(conv_1)                              # pool_1: 14 x 14 x 32
    
    with tf.variable_scope("conv_2"):
        conv_2 = conv2d(pool_1, [5, 5, 32, 64], [64])          # conv_2: 14 x 14 x 64
        pool_2 = max_pool(conv_2)                              # pool_2: 7 x 7 x 64
        
    with tf.variable_scope("fc"):
        pool_2_flat = tf.reshape(pool_2, [-1, 7 * 7 * 64])     # from pool_2
        #pool_2_flat = tf.layers.flatten()                       # similar with the above line
        fc_1 = layer(pool_2_flat, [7 * 7 * 64, 1024], [1024])   # fc_1: 1024
        
        # dropout
        fc_1_dropout = tf.nn.dropout(fc_1, keep_prob=keep_prob)
        
    with tf.variable_scope("out"):
        out = layer(fc_1_dropout, [1024, 10], [10])
    
    return out

In [8]:
def training(cost, global_step):
    tf.summary.scalar("cost", cost)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=9e-1, beta2=0.999, epsilon=1e-8, name="Adam")
    train_opt = optimizer.minimize(cost, global_step=global_step)
    return train_opt

In [9]:
def evaluate(output, y, name):
    compare = tf.equal(tf.argmax(output, axis=1), tf.argmax(y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(compare, tf.float32))
    tf.summary.scalar("eval/{}".format(str(name)), accuracy)
    return accuracy

## Flow

In [10]:
full_training = True
learning_rate = 1e-2
training_epochs = 1000 if full_training else 1
batch_size = 100 if full_training else 1000
display_step = 1000 if full_training else 100

In [11]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/jiankaiwang/Desktop/devops/tmp/MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [12]:
with tf.Graph().as_default():
    with tf.variable_scope("cnn_model") as scope:
        x = tf.placeholder("float", [None, 784])
        y = tf.placeholder("float", [None, 10])
        
        output = inference(x)
        cost = loss(output=output, y=y)
        global_step = tf.Variable(initial_value=0, name="global_step", trainable=False)
        train_opt = training(cost=cost, global_step=global_step)
        eval_opt_val = evaluate(output=output, y=y, name="valid")
        eval_opt_test = evaluate(output=output, y=y, name="test")
        
        init_var = tf.global_variables_initializer()
        summary_opt = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            
            model_path = "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn/"
            ckpt_log = os.path.join(model_path, "checkpoint")
            if os.path.isdir(model_path) and os.path.isfile(ckpt_log):
                #ckpt_path = "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn/model-checkpoint-{}".format(restrore_idx)
                ckpt_path = ""
                with open(ckpt_log, "r") as fin:
                    for line in fin:
                        ckpt_path += line.strip()   # only read the first line
                        break
                # model_checkpoint_path: "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn/model-checkpoint-1020"
                ckpt_path = ckpt_path[24:-1]
                
                if os.path.isfile(ckpt_path + ".meta"):
                    scope.reuse_variables()
                    var_list_opt = ["conv_1/W", "conv_1/b", "conv_2/W", "conv_2/b", "fc/W", "fc/b", "out/W", "out/b"]
                    var_list_opt = [tf.get_variable(v) for v in var_list_opt]
                    saver.restore(sess, ckpt_path)
                    print("Load pretrained.")
                else:
                    # no need to initialize if loading pretrained data
                    sess.run(init_var)
            else:
                # no need to initialize if loading pretrained data
                sess.run(init_var)
            
            summary_writer = tf.summary.FileWriter(\
                    "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn", \
                    graph=sess.graph)
            
            for epoch in range(training_epochs):
                avg_cost = 0.
                total_batch = int(mnist.train.num_examples / batch_size)
                
                for idx in range(total_batch):
                    b_x, b_y = mnist.train.next_batch(batch_size=batch_size)
                    
                    feed_dict_data = {x: b_x, y: b_y}
                    sess.run(train_opt, feed_dict=feed_dict_data)
                    
                    batch_cost = sess.run(cost, feed_dict=feed_dict_data)
                    avg_cost += (batch_cost / total_batch)
                    
                if epoch % display_step == 0:
                    feed_dict_val_data = {x: mnist.validation.images, y: mnist.validation.labels}
                    acc = sess.run(eval_opt_val, feed_dict=feed_dict_val_data)
                    step = sess.run(global_step)
                    print("Step:", step, "Accuracy:", acc, "Vaildation Error:", (1-acc))
                    tf.summary.scalar("Validation_accuracy", acc)

                    summary_str = sess.run(summary_opt, feed_dict = feed_dict_data)
                    summary_writer.add_summary(summary_str, step)

                    saver.save(sess, \
                               "/Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn/model-checkpoint", \
                              global_step=global_step)
            
            print("Training finished.")
            
            feed_dict_test_data = {x: mnist.test.images, y: mnist.test.labels}
            acc = sess.run(eval_opt_test, feed_dict=feed_dict_test_data)
            print("Test accuracy:", acc)

INFO:tensorflow:Restoring parameters from /Users/jiankaiwang/Desktop/devops/tmp/basic_tf/basic_cnn/model-checkpoint-1017
Load pretrained.
Step: 1018 Accuracy: 0.7738 Vaildation Error: 0.22619998455047607
Step: 1019 Accuracy: 0.7694 Vaildation Error: 0.2305999994277954
Step: 1020 Accuracy: 0.7722 Vaildation Error: 0.22780001163482666


KeyboardInterrupt: 

# Image Pre-processing 

```python
tf.image.per_image_standardization(image)
```

# Image Data Augmentation

```python
tf.random_crop()
tf.image.transpose_image()
tf.image.random_flip_up_down()
tf.image.random_flip_left_right()
tf.image.random_brightness()
tf.image.random_contrast()
tf.image.random_saturation()
tf.image.random_hue()
```

# Batch Normalizarion

In [13]:
from tensorflow.python.ops import control_flow_ops

## BN for Convolutional Layer

In [28]:
# example for control_flow_ops.cond
# similar with if-else

a = tf.constant(1)
b = tf.constant(10)

condition = tf.convert_to_tensor(False, dtype='bool')
test = control_flow_ops.cond(condition, lambda: a, lambda: b)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(test))

10


In [14]:
def conv_batch_norm(x, n_out, phase_train):
    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.get_variable("alpha", [n_out], initializer=gamma_init)
    
    batch_mean, batch_var = tf.nn.moments(x, len(x.shape())-1, name='moments')
    
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    
    mean, var = control_flow_ops.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    normed = tf.nn.batch_norm_with_global_normalization(x, mean, var, beta, gamma, 1e-3, True)
    return normed

In [16]:
def BN_conv2d(input, weight_shape, bias_shape, phase_train):
    in_count = weight_shape[0] * weight_shape[1] * weight_shape[2]
    weight_init = tf.random_normal_initializer(stddev=(2.0/in_count)**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    conv_out = tf.nn.conv2d(input, W, strides=[1,1,1,1], padding="SAME")
    logits = tf.nn.bias_add(conv_out, b)
    return tf.nn.relu(conv_batch_norm(logits, weight_shape[3], phase_train))

## BN for Non-Convolutional Layer

In [15]:
def layer_batch_norm(x, n_out, phase_train):
    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    gamma = tf.get_variable("alpha", [n_out], initializer=gamma_init)
    
    batch_mean, batch_var = tf.nn.moments(x, len(x.shape())-1, name='moments')
    
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
    
    mean, var = control_flow_ops.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    x_r = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(x_r, mean, var, beta, gamma, 1e-3, True)
    return tf.reshape(normed, [-1, n_out])

In [17]:
def BN_layer(input, weight_shape, bias_shape, phase_train):
    in_count = weight_shape[0] * weight_shape[1]
    w_init = tf.random_normal_initializer(stddev=(2.0/in_count)**0.5)
    b_init = tf.constant_initializer(value=0)
    W = tf.get_variable("W", weight_shape, initializer=w_init)
    b = tf.get_variable("b", bias_shape, initializer=b_init)
    logits = tf.matmul(input, W) + b
    return tf.nn.relu(layer_batch_norm(logits, weight_shape[1], phase_train))